In [ ]:
import ffmpeg
import numpy as np
import cv2

# Function to apply Hough Transform to a frame
def apply_hough_transform(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply edge detection (Canny)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    
    # Detect lines using Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 350)
    
    # Make the frame writable
    frame = frame.copy()

    # Draw lines on the original frame
    if lines is not None:
        for rho, theta in lines[:, 0]:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
    return frame

# Input and output video file paths
input_video_path = "dataset/MIDItest/miditest_videos/5.mp4"
output_video_path = "output_folder/output_video.mp4"

# Probe video dimensions
probe = ffmpeg.probe(input_video_path)
video_stream = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
width = int(video_stream['width'])
height = int(video_stream['height'])

# Read video frames with FFmpeg
process = (
    ffmpeg.input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='bgr24')
    .run_async(pipe_stdout=True)
)

# Prepare FFmpeg output process
output_process = (
    ffmpeg.input('pipe:', format='rawvideo', pix_fmt='bgr24', s=f'{width}x{height}', r=30)
    .output(output_video_path, vcodec='libx264', pix_fmt='yuv420p')
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

while True:
    # Read frame from input video
    in_bytes = process.stdout.read(width * height * 3)
    if not in_bytes:
        break

    # Convert bytes to a numpy array
    frame = (
        np.frombuffer(in_bytes, np.uint8)
        .reshape([height, width, 3])
    )

    # Apply Hough Transform
    transformed_frame = apply_hough_transform(frame)

    # Write frame to output video
    output_process.stdin.write(transformed_frame.tobytes())

# Release resources
process.stdout.close()
process.wait()
output_process.stdin.close()
output_process.wait()
cv2.destroyAllWindows()
